# 如何在子链之间路由 How to route between sub-chains

路由允许您创建非确定性链，其中上一步的输出定义下一步。路由可以帮助您定义状态并使用与这些状态相关的信息作为模型调用的上下文，从而帮助提供与模型交互的结构和一致性。

有两种方法可以执行路由：

 - 有条件地从 RunnableLambda 返回可运行对象（推荐）
 - 使用 RunnableBranch（旧版）


In [2]:
import os
from dotenv import find_dotenv,load_dotenv

_ = load_dotenv(find_dotenv())

In [9]:
from langchain_community.chat_models import ChatZhipuAI,ChatBaichuan
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """给出以下用户问题，将其分类为关于“zhipu”、“baichuan”或“其他”。

        不要用多个词回答。
        
        <question>
        {question}
        </question>
        
        分类："""
    )
    | ChatZhipuAI(model="glm-4")
    | StrOutputParser()
)

chain.invoke({"question": "如何调用zhipu?"})

'zhipu'

In [10]:
zhipu_chain = (
    PromptTemplate.from_template("""您是 zhipu 方面的专家。\
        回答问题时，请始终以“正如张三告诉我的那样”开头。\
        回答以下问题：
        
        问题：{question}
        答案：
    """)
    | ChatZhipuAI(model="glm-4")
    | StrOutputParser()
)

baichuan_chain = (
    PromptTemplate.from_template("""您是 baichuan 方面的专家。\
        回答问题时，请始终以“正如王武告诉我的那样”开头。\
        回答以下问题：
        
        问题：{question}
        答案：
    """)
    | ChatBaichuan()
    | StrOutputParser()
)

general_chain = (
    PromptTemplate.from_template("""您是general方面的专家。\
        回答问题时，请始终以“正如王武告诉我的那样”开头。\
        回答以下问题：
        
        问题：{question}
        答案：
    """)
    | ChatZhipuAI(model="glm-4")
    | StrOutputParser()
)

## 使用自定义函数（推荐）
您还可以使用自定义函数在不同输出之间进行路由。以下是示例：

In [15]:
def route(info):
    print("=====",info)
    if "zhipu" in info["topic"].lower():
        return zhipu_chain
    elif "baichuan" in info["topic"].lower():
        return baichuan_chain
    else:
        return general_chain

In [22]:
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic":chain,"question":lambda x: x["question"]} 
    | RunnableLambda(route)
)

In [25]:
full_chain.invoke({"question": "如何使用zhipu?"})

===== {'topic': 'zhipu', 'question': '如何使用zhipu?'}


'正如张三告诉我的那样，使用Zhipu（智谱）的方法主要取决于你想要达到的目标。Zhipu是一款在线协作工具，可以用于项目管理、任务分配、团队协作等。以下是使用Zhipu的基本步骤：\n\n1. 注册账户：首先，你需要访问Zhipu的官方网站并注册一个账户。\n\n2. 创建项目：登录后，你可以创建一个新的项目，并在其中添加任务列表。\n\n3. 添加任务：在每个任务列表中，你可以添加具体的任务，并且可以给每个任务设定优先级、截止日期和分配给团队成员。\n\n4. 团队协作：你可以邀请你的同事或团队成员加入你的项目，共同协作完成任务。\n\n5. 进度追踪：任务开始执行后，你可以通过Zhipu跟踪任务进度，更新任务状态。\n\n6. 交流沟通：Zhipu提供了评论区功能，项目成员可以在任务下面留言交流，保持信息同步。\n\n7. 文件共享：你可以上传文件到任务或项目，以便团队成员共享和下载。\n\n8. 定期回顾：定期查看项目的整体进度，对任务进行回顾和调整。\n\n正如张三告诉我的那样，使用Zhipu的关键是要保持团队的沟通和协作，充分利用其提供的各项功能，从而高效地完成项目。'